In [1]:
import pandas as pd
import matplotlib
import pandas_profiling

In [5]:
df1 = pd.read_csv('/Users/frankdezwart/code/molecular_properties/data/raw/train.csv')

In [7]:
pandas_profiling.ProfileReport(df1)

Number of variables,6
Number of observations,4658147
Total Missing (%),0.0%
Total size in memory,213.2 MiB
Average record size in memory,48.0 B
Numeric,4
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [11]:
df2 = pd.read_csv('/Users/frankdezwart/code/molecular_properties/data/raw/structures.csv')

In [12]:
pandas_profiling.ProfileReport(df2)

Number of variables,6
Number of observations,2358657
Total Missing (%),0.0%
Total size in memory,108.0 MiB
Average record size in memory,48.0 B
Numeric,4
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0
